In [1]:
from data_loader import TimeWindowSegmenter


In [2]:
data_processor = TimeWindowSegmenter(
  df_path="wsidm.parquet",
  window_size=10,   # 10 sekund
  step_size=10,     # 10 sekund - ramki się nie pokrywają w danych
)

fix timestamps
end of fixing timestamps


In [3]:
import numpy as np
from scipy.signal import welch
from scipy.fft import fft

In [4]:
def dominant_frequency(signal, fs):
    """
    Dominująca częstotliwość to ta składowa częstotliwościowa sygnału, 
    która posiada największą moc (czyli największą amplitudę w widmie mocy). 
    Jest to częstotliwość, która dominuje energetycznie w analizowanym odcinku czasu.
    """
    freqs, psd = welch(signal, fs, nperseg=len(signal))
    dom_freq = freqs[np.argmax(psd)]
    return dom_freq

In [5]:
def spectral_entropy(signal, fs):
    """
     Entropia widmowa to miara nieuporządkowania lub losowości rozkładu widmowego sygnału. 
     Bazuje na teorii informacji (entropii Shannona) i opisuje, jak równomiernie rozłożona 
     jest energia sygnału w domenie częstotliwości.

     - Niska entropia oznacza, że energia jest skoncentrowana wokół kilku częstotliwości — sygnał jest bardziej "uporządkowany".
     - Wysoka entropia sugeruje rozproszenie energii po wielu częstotliwościach — sygnał jest bardziej "chaotyczny"
    """
    _, psd = welch(signal, fs, nperseg=len(signal))
    psd_norm = psd / np.sum(psd)  # normalizacja
    entropy = -np.sum(psd_norm * np.log2(psd_norm + 1e-12))  # dodajemy epsilon żeby uniknąć log(0)
    entropy /= np.log2(len(psd_norm))  # normalizacja do [0,1]
    return entropy

In [6]:
def spectral_energy(signal):
    """
    Energia widmowa to całkowita suma energii zawarta w widmie sygnału. 
    Można ją rozumieć jako "siłę" sygnału rozłożoną w dziedzinie częstotliwości.
    
    - Pomaga ocenić ogólną intensywność sygnału — im większa energia, tym silniejszy 
    jest sygnał w danym zakresie czasu.
    """
    spectrum = np.abs(fft(signal))**2
    energy = np.sum(spectrum) / len(spectrum) # normalizujemy do długości sygnału
    return energy

In [7]:
def spectral_bandwidth(signal, fs):
    """
    Szerokość pasma — odchylenie standardowe widma względem środka ciężkości (centroidu).

    - Małe pasmo sugeruje, że sygnał zawiera głównie wąski zakres częstotliwości.
    - Duże pasmo oznacza obecność wielu różnych składowych częstotliwościowych — np. złożone sygnały biologiczne.
    """
    freqs, psd = welch(signal, fs, nperseg=len(signal))
    psd_norm = psd / np.sum(psd)
    centroid = np.sum(freqs * psd_norm)
    bandwidth = np.sqrt(np.sum(((freqs - centroid) ** 2) * psd_norm))
    return bandwidth

In [8]:
def extract_features_from_window(window, fs=20):  # 200 próbek = 10s -> fs = 20 Hz
    features = []
    axes = ['ac_x', 'ac_y', 'ac_z', 'g_x', 'g_y', 'g_z']
    for axis in axes:
        signal = window[axis].astype(float).values
        features.extend([
            dominant_frequency(signal, fs),
            spectral_entropy(signal, fs),
            spectral_energy(signal),
            spectral_bandwidth(signal, fs),
        ])
    return features

In [10]:
X = []
Y = []
for window in data_processor.segment():
    features = extract_features_from_window(window)
    label = window['Activity Label'].mode()[0]
    X.append(features)
    Y.append(label)

Segmenting: 100%|██████████| 250/250 [00:08<00:00, 29.96it/s]
